# Comparing linear regression and DFL-IO

Steps comparing the linear and inverse-DFL models:
1. Save two datasets
4. Split each dataset into train and test sets
5. Run DFL-IO with each training set, get weights $\theta^{\text{IO,close}}$ and $\theta^{\text{IO,wide}}$
6. Run linear regression on training set, get weights $\theta^{\text{R,close}}$ and $\theta^{\text{R,wide}}$
7. Compare all models on each training set, for each model:
    1. Predict demand using weights and features
    2. Solve MCFND with demand, get design variables $\hat{y}$
    3. Solve MCF-Flow with fixed design variables $\hat{y}$, get $\hat{x}$
    4. Compare costs of $\hat{y}, \hat{x}$ with optimal $x^*, y^*$ 

## Imports and configurations

In [ ]:
using JuMP, Gurobi
using LinearAlgebra
using Distributions, Random, PDMats
using MLJ, Tables
using DataFrames, DataFramesMeta
using JLD, CSV
using PlotlyJS
using Pipe
using LaTeXStrings

Random.seed!(42)

In [ ]:
using Revise

includet("../models/forward.jl")
import .Forward as Forward

includet("../models/inversedemand.jl")
import .InverseDemand as IODemand

includet("../models/inverselinreg.jl")
import .InverseLinReg as IOLinReg

includet("../datagen/data-generation.jl")
import .DataGeneration as DataGen

In [ ]:
Linear = @MLJ.load LinearRegressor pkg = "MLJLinearModels"

In [ ]:
BASE_DATA_PATH = "../data/"
BASE_RESULTS_PATH = "../results/"

CLOSE_DATA_NAME = "close"
WIDE_DATA_NAME = "wide"

function dataset_path(n_points)
    return joinpath(BASE_DATA_PATH, "data_$n_points.jld") 
end

function results_path(n_points)
    return joinpath(BASE_RESULTS_PATH, "results_$n_points.csv")
end

## Problem parameters

Make a smaller problem with 1 commodity and 2 possible arcs, one low-ish capacity ($C$) and one high ($\infty$)


In [ ]:
forward_params = Forward.Params(
    n_paths=2, 
    n_commodities=1,
    capacities=[100, 100000],
    design_costs=[10, 100],
    flow_costs=[10 100]',
    enabled_flows=ones(Bool, (2, 1))
)

datagen_params = DataGen.DataGenParams(
    weights=[1.5 -3 2], 
    noise_variance=[5.0^2]
)

inverse_params = IOLinReg.Params(
    n_features=datagen_params.n_features, 
    forward_params=forward_params, 
    with_noise=true
)

## Data generation

Generate two datasets using fixed weights $\Theta$:
- $\mathcal{D}_{\text{close}}$ with $\mathbb{E}[d] = C$ 
- $\mathcal{D}_{\text{wide}}$ with $\mathbb{E}[d] \ll C$. 

Procedure for each data point in the dataset:
1. Draw $\phi_1, \ldots, \phi_{m-1} \sim U(a, b)$ for some $a, b$
2. Set $\phi_m$ such that $\sum_{i=1}^m \theta_i \phi_i = \mathbb{E}[d]$
3. Draw noise $\epsilon \sim \mathcal{N}(0, \sigma^2)$ and compute $d = \sum_{i=1}^m \theta_i \phi_i + \epsilon$
4. Solve MCFND for $d$ 
5. Datapoint $(\phi, d, x^*, y^*) \in \mathcal{D}$

Repeat dataset creation for several number of points


In [ ]:
n_points = [100, 1000, 3000, 10000]
close_target_demand = 100
wide_target_demand = 20

gurobi_env = Gurobi.Env()

In [ ]:
for n in n_points
    close_dataset = DataGen.generate_dataset(forward_params, datagen_params, n_points=n, target_demand=close_target_demand, gurobi_env=gurobi_env)
    wide_dataset = DataGen.generate_dataset(forward_params, datagen_params, n_points=n, target_demand=wide_target_demand, gurobi_env=gurobi_env)

    save(dataset_path(n), CLOSE_DATA_NAME, close_dataset, WIDE_DATA_NAME, wide_dataset, compress=true)
end

## Model training

Define a training function and a prediction function for each model:
- `train_{model_type}_model` takes in a training dataset of `IOLinReg.SolutionPoint`s, and returns a trained model
- `predict_{model_type}_model` takes a model of the correct type and a test dataset of `IOLinReg.SolutionPoint`s, and returns a vector of predicted demands

Utility functions

In [ ]:
function load_dataset(n_points)
    dataset = JLD.load(dataset_path(n_points))

    return dataset[CLOSE_DATA_NAME], dataset[WIDE_DATA_NAME]
end

function convert_dataset_to_mlj(dataset)
    features = DataFrame(vcat(map(sol -> sol.linreg_features', dataset)...), :auto)
    demands = vcat(map(sol -> sol.actual_demands, dataset)...)

    return features, demands
end

DFL-IO model training and prediction functions

In [ ]:
function train_inverse_model(training_dataset)
    model = IOLinReg.create_problem(inverse_params, training_dataset, gurobi_env=gurobi_env)

    return IOLinReg.solve_problem!(model, inverse_params)
end

function predict_inverse_model(inverse_solution, test_dataset)
    features = map(row -> row.linreg_features, test_dataset)
    predict = f -> IOLinReg.predict_inverse_model(inverse_solution, f)

    return vcat(map(predict, features)...)
end

Linear regression model training and prediction functions

In [ ]:
function train_linear_model(training_dataset)
    features, demands = convert_dataset_to_mlj(training_dataset)

    model = Linear()
    mach = machine(model, features, demands)
    fit!(mach)

    return mach
end

function predict_linear_model(linreg_machine, test_dataset)
    features, _ = convert_dataset_to_mlj(test_dataset)
    return predict(linreg_machine, features)
end

## Model evaluation

Define an evaluation procedure `evaluate_model_on_dataset` as described in point 5 of the introduction. Takes a train and test dataset, and a training and prediction function for a given model.

In [ ]:
function evaluate_model_on_dataset(train_data, test_data, train_model, make_predictions; gurobi_env=nothing)  
    trained_model = train_model(train_data)
    predicted_demands = make_predictions(trained_model, test_data)
    designed_network = compute_predicted_network_design(forward_params, predicted_demands, gurobi_env=gurobi_env)

    return compute_flow_problem_results(forward_params, test_data, designed_network, predicted_demands)
end

function compute_predicted_network_design(forward_params, predicted_demands; gurobi_env=nothing)
    solve_mcfnd = d -> Forward.create_and_solve_problem(forward_params, d, silent=true, gurobi_env=gurobi_env)
    return map(d -> solve_mcfnd(d).z_sol, predicted_demands) 
end

function compute_flow_problem_results(forward_params, test_dataset, predicted_z_sols, predicted_demands; gurobi_env=gurobi_env)
    actual_demands = map(row -> row.actual_demands, test_dataset)
    solve_flow = (d, z_sol) -> Forward.create_and_solve_flow_problem(forward_params, d, z_sol, silent=true, gurobi_env=gurobi_env)
    forward_solutions = map(solve_flow, actual_demands, predicted_z_sols)
    
    task_losses = map(sol -> sol.objective_value, forward_solutions)
    recourse_flow = map(sol -> sol.recourse_flow, forward_solutions)
    
    return DataFrame(
        task_loss=task_losses, 
        recourse_flow=recourse_flow, 
        predicted_demand=predicted_demands, 
        actual_demand=map(first, actual_demands)
    )
end

### Model comparison pipeline and result cleaning

For a given full dataset of `n_points`, obtain the results of the DFL-IO and the Linear Regression model over the close and wide datasets

In [ ]:
function compare_models(n_points; test_train_split=0.7)
    close_dataset, wide_dataset = load_dataset(n_points)

    close_train, close_test = partition(close_dataset, test_train_split)
    wide_train, wide_test =  partition(wide_dataset, test_train_split)

    close_io_results = evaluate_model_on_dataset(close_train, close_test, train_inverse_model, predict_inverse_model, gurobi_env=gurobi_env)
    close_linreg_results = evaluate_model_on_dataset(close_train, close_test, train_linear_model, predict_linear_model, gurobi_env=gurobi_env)
    wide_io_results = evaluate_model_on_dataset(wide_train, wide_test, train_inverse_model, predict_inverse_model, gurobi_env=gurobi_env)
    wide_linreg_results = evaluate_model_on_dataset(wide_train, wide_test, train_linear_model, predict_linear_model, gurobi_env=gurobi_env)

    return vcat(
        specify_model_and_data(close_io_results, "close", "io"),
        specify_model_and_data(close_linreg_results, "close", "linreg"),
        specify_model_and_data(wide_io_results, "wide", "io"),
        specify_model_and_data(wide_linreg_results, "wide", "linreg")
    )
end


function specify_model_and_data(results_data, data_type, model_type)
    length = nrow(results_data)

    data_column = categorical(fill(data_type, length))
    model_column = categorical(fill(model_type, length))

    types_df = DataFrame(dataset=data_column, model=model_column)

    return hcat(results_data, types_df)
end

Generating and storing the results

In [ ]:
all_results = [n => compare_models(n) for n in n_points]

for (n, res) in all_results
    CSV.write(results_path(n), res)
end;

In [ ]:
combined_results = vcat([
    hcat(res, DataFrame(:n_points => fill(n, nrows(res))))
    for (n, res) in all_results
]...)

CSV.write(results_path("all"), combined_results)

## Analysis

In [ ]:
function load_results(n_points)
    results = CSV.read(results_path(n_points), DataFrame)

    if "n_points" ∉ names(results)
        @transform!(results, :n_points = fill("$n_points", nrow(results)))
    end

    @transform!(results, :dataset = categorical(:dataset))
    @transform!(results, :model = categorical(:model))
    @transform!(results, :n_points = string.(:n_points))

    return results
end

results = load_results("all")
subresults = [n => load_results(n) for n in n_points]

first(results, 5)

### Recouse paths and model robustness

No point in the wide dataset ever uses recourse flow, so we filter that dataset out

In [ ]:
results[results.dataset .== "wide" .&& results.recourse_flow .> 0, :]

In [ ]:
function analyze_n_recourse(res)
    return @chain res begin 
        groupby([:n_points, :dataset, :model])
        subset(:dataset => d -> d .!= "wide", ungroup=false)
        combine(:recourse_flow => (r -> count(x -> x .> 0, r)) => :n_recourse,
                :recourse_flow => (r -> count(x -> x .> 0, r) / nrows(r)) => :frac_recourse,
                :recourse_flow => mean)
        sort(:n_points, by=n -> parse(Int, n))
        @transform(:pct_recourse = 100 .* :frac_recourse)
        @transform(:str_pct_recourse = string.(round.(:pct_recourse, digits=1), "%"))
    end
end

recourse_analysis = analyze_n_recourse(results)

First analyse, for every model and dataset, how many predicted demands result in having to use the recourse path. We are testing the robustness of the prediction algorithm

In [ ]:
plot(
    recourse_analysis,
    kind="bar",
    y=:pct_recourse,
    text=:str_pct_recourse,
    facet_col=:n_points, 
    color=:model,
    Layout(
        xaxis_title="",
        yaxis_title="% using recourse path",
        title="Percentage of predicted demands that need to use recourse path, compared between DFL-IO and Linear Regression"
    )
)

We then look at the distribution of the flow over the recourse path for each model

In [ ]:
all_recourses = @chain results begin
    subset(:recourse_flow => f -> f .> 0)
    subset(:dataset => d -> d .!= "wide")
end

first(all_recourses, 5)

In [ ]:
plot(
    all_recourses,
    kind="histogram",
    x=:recourse_flow,
    facet_col=:n_points, 
    color=:model,
    opacity=0.5,
    Layout(
        xaxis_title="Amount of recourse flow",
        yaxis_title="# of occurences",
        title="Distribution of recourse flow amount for DFL-IO and Linear Regression models",
        barmode="overlay"
    )
)

### Task losses

In [ ]:
@chain results begin
    subset(:recourse_flow => r -> r .<= 0)
    groupby([:n_points, :dataset, :model])
    combine(:task_loss => (x -> [(mean(x), median(x), extrema(x)...)]) => [:mean, :median, :min, :max])
end

In [ ]:
plot(
    subset(results, :recourse_flow => r -> r .<= 0),
    kind="box",
    x=:task_loss,
    facet_row=:n_points,
    facet_col=:dataset,
    color=:model,
    Layout(
        height=1000,
        title="Comparison of task losses between IO and Linear Regression, by dataset type and #points",
        #xaxis_title="task_loss"
    )
)

In [ ]:
function plot_task_loss_distributions(results, dataset)
    no_recourse_results = @chain results begin
        subset(:recourse_flow => r -> r .<= 0)
        subset(:dataset => d -> d .== dataset)
    end
    
    return plot(
        no_recourse_results,
        kind="histogram",
        x=:task_loss,
        facet_row=:n_points, 
        color=:model,
        opacity=0.5,
        Layout(
            xaxis_title="",
            yaxis_title="",
            title="",
            barmode="overlay",
            height="1000",
            width="700"
        )
    )
end

plot_task_loss_distributions(results, "close")

In [ ]:
plot_task_loss_distributions(results, "wide")

### Comparison of predicted demands

In [ ]:
pred_io = @chain results begin
    subset(:model => m -> m .== "io")
    @rename(:pred_io = :predicted_demand, :recourse_io = :recourse_flow)
    select(:dataset, :n_points, :pred_io, :recourse_io)
end 

pred_linreg = @chain results begin
    subset(:model => m -> m .== "linreg")
    @rename(:pred_linreg = :predicted_demand, :recourse_linreg = :recourse_flow)
    select(:pred_linreg, :recourse_linreg)
end

pred_compared = hcat(pred_io, pred_linreg)

first(pred_compared, 5)

In [ ]:
function categorise_compared_recourse(recourse_flow_io, recourse_flow_linreg)
    if recourse_flow_io .> 0 && recourse_flow_linreg .> 0
        return "both"
    elseif recourse_flow_io .> 0
        return "io"
    elseif recourse_flow_linreg > 0
        return "linreg"
    end
    
    return "none"
end

function plot_prediction_comparison(pred_compared, n_points, dataset)
    filtered = @chain pred_compared begin
        @subset(:n_points .== "$n_points")
        @subset(:dataset .== dataset)
        @transform(@byrow :recourse_use = categorise_compared_recourse(:recourse_io, :recourse_linreg))
    end

    mn, mx = extrema(vcat(filtered.pred_io, filtered.pred_linreg))
    t = mn:0.01:mx

    return plot(
        [
            scatter(x=t, y=t, mode="lines", name="Identity"),
            scatter(filtered[filtered.recourse_use .== "none", :], x=:pred_linreg, y=:pred_io, mode="markers", name="No model uses recourse"),
            scatter(filtered[filtered.recourse_use .== "io", :], x=:pred_linreg, y=:pred_io, mode="markers", name="Only DFL-IO uses recourse"),
            scatter(filtered[filtered.recourse_use .== "linreg", :], x=:pred_linreg, y=:pred_io, mode="markers", name="Only LinReg uses recourse"),
            scatter(filtered[filtered.recourse_use .== "both", :], x=:pred_linreg, y=:pred_io, mode="markers", name="Both models use recourse")
        ],
        Layout(
            xaxis_title="Predicted demand (Linear Regression)",
            yaxis_title="Predicted demand (DFL-IO)"
        )
    )
end

plot_prediction_comparison(pred_compared, 100, "close")

In [ ]:
plot_prediction_comparison(pred_compared, 1000, "close")

In [ ]:
plot_prediction_comparison(pred_compared, 3000, "close")

In [ ]:
plot_prediction_comparison(pred_compared, 10000, "close")

In [ ]:
plot_prediction_comparison(pred_compared, 100, "wide")

In [ ]:
plot_prediction_comparison(pred_compared, 1000, "wide")

In [ ]:
plot_prediction_comparison(pred_compared, 3000, "wide")

In [ ]:
plot_prediction_comparison(pred_compared, 10000, "wide")